In [1]:
!pip install -q datasets transformers sentencepiece wandb

In [ ]:
import wandb
wandb.login()

%env WANDB_PROJECT=stock_QA

In [33]:
from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token


In [23]:
!git config --global credential.helper store

In [4]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.3.4-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


# Fine-tuning a Albert

In [6]:
# This flag is the difference between SQUAD v1 or 2 (if you're using another dataset, it indicates if impossible
# answers are allowed or not).
squad_v2 = False
model_checkpoint = "m3hrdadfi/albert-fa-base-v2"
batch_size = 16

## Loading the dataset

In [7]:
from datasets import load_dataset, load_metric

datasets = load_dataset('SajjadAyoubi/persian_qa')
datasets

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

/root/.cache/huggingface/datasets/downloads/6bed7cf0f82631fb5231ba39906ba835ef73fb005b6e7371869afd01e5f98ddc


Generating validation split: 0 examples [00:00, ? examples/s]

/root/.cache/huggingface/datasets/downloads/3b640ed1f99bd004e2d515cf66938e75cd85d8f359739d9ba644ed3dc533b849
Dataset persian_qa downloaded and prepared to /root/.cache/huggingface/datasets/SajjadAyoubi___persian_qa/persian_qa/1.0.0/adcc9e82d1a679ba85f7958663d8e771894c35e2fbc6a92d9ea2b6a8a72f9225. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 9008
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 930
    })
})

In [14]:
datasets["train"][0]

{'answers': {'answer_start': [114], 'text': ['در شرق شهر مبارکه']},
 'context': 'شرکت فولاد مبارکۀ اصفهان، بزرگ\u200cترین واحد صنعتی خصوصی در ایران و بزرگ\u200cترین مجتمع تولید فولاد در کشور ایران است، که در شرق شهر مبارکه قرار دارد. فولاد مبارکه هم\u200cاکنون محرک بسیاری از صنایع بالادستی و پایین\u200cدستی است. فولاد مبارکه در ۱۱ دوره جایزۀ ملی تعالی سازمانی و ۶ دوره جایزۀ شرکت دانشی در کشور رتبۀ نخست را بدست آورده\u200cاست و همچنین این شرکت در سال ۱۳۹۱ برای نخستین\u200cبار به عنوان تنها شرکت ایرانی با کسب امتیاز ۶۵۴ تندیس زرین جایزۀ ملی تعالی سازمانی را از آن خود کند. شرکت فولاد مبارکۀ اصفهان در ۲۳ دی ماه ۱۳۷۱ احداث شد و اکنون بزرگ\u200cترین واحدهای صنعتی و بزرگترین مجتمع تولید فولاد در ایران است. این شرکت در زمینی به مساحت ۳۵ کیلومتر مربع در نزدیکی شهر مبارکه و در ۷۵ کیلومتری جنوب غربی شهر اصفهان واقع شده\u200cاست. مصرف آب این کارخانه در کمترین میزان خود، ۱٫۵٪ از دبی زاینده\u200cرود برابر سالانه ۲۳ میلیون متر مکعب در سال است و خود یکی از عوامل کم\u200cآبی زاینده\u200cرود شناخته می\u

We can see the answers are indicated by their start position in the text (here at character 515) and their full text, which is a substring of the context as we mentioned above.

To get a sense of what the data looks like, the following function will show some examples picked randomly in the dataset (automatically decoding the labels in passing).

In [10]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(datasets["train"])

,id,title,context,question,answers
0,7432,پروستات,پروستات یا پروفسرته عضوی از دستگاه تناسلی مردانه است و به اندازه و شکل یک گردوی کوچک، در ابتدا مجرای ادراری در لگن خاصره قرار دارد. پروستات غده‌ای است سفت به شکل یک هرم وارونه که قسمتی از آن بافت غددی و قسمتی بافت عضلانی – همبندی دارد و مجرای ادرار پروستاتی را احاطه می‌نماید. غده پروستات در موقع انزال منقبض می‌شود و ماده شیری رنگ قلیایی به منی اضافه می‌کند. حالت ژلاتینی منی به‌دلیل همین اتفاق است. ترشحات پروستات در انسان به‌طور تقریبی و معمولی حدود ۳۰ درصد حجمی منی را تشکیل می‌دهد. غدهٔ پروستات در سراسر دوران کودکی نسبتاً کوچک باقی می‌ماند و در سن بلوغ تحت تأثیر تستوسترون شروع به رشد می‌کنند. این غده در حدود سن ۲۰ سالگی به یک اندازهٔ تقریباً ثابت می‌رسد و تا سن حدود ۴۰ تا ۵۰ سالگی به همین اندازه (حدود ۳۰ سی سی) باقی می‌ماند. وزن پروستات در فرد بالغ سالم حدود ۲۰–۲۵ گرم و اندازه آن ۴×۲×۳ سانتی‌متر است. پروستات در تقسیم‌بندی غدد بدن به درون‌ریز و برون‌ریز، به دلیل اینکه مواد ترشحی آن از راه مجاری به بیرون از بدن می‌ریزد، جز غدد برون‌ریز محسوب می‌شود.,ظاهر پروستات چگونه است؟؟,"{'text': ['یک گردوی کوچک'], 'answer_start': [71]}"
1,6605,فوتبال,فوتبال یک ورزش تیمی و محبوب‌ترین ورزش در بیشتر کشورهای جهان است. فوتبال را دو تیم یازده‌نفره متشکل از بازیکنان فوتبال با یک توپ فوتبال بر روی چمن طبیعی یا مصنوعی با هدف گل‌زدن انجام می‌دهند. یازده بازیکن هر تیم شامل چند مهاجم، هافبک، مدافع و یک دروازه‌بان می‌شوند که توسط سرمربی در زمین چیده می‌شوند و مانند یک سیستم عمل می‌کنند که سیستم فوتبال نامیده می‌شود. امروزه بیش از ۲۵۰ میلیون فوتبالیست در بیش از ۲۰۰ کشور وجود دارد. قدمت فوتبال به چندین سدهٔ پیش از میلاد و به چین بازمی‌گردد که به تدریج در یونان و روم نیز رواج می‌یابد. فوتبال مدرن امروزی نیز در سال ۱۸۶۶ در مدارس انگلستان شکل‌گرفت و به تدریج گسترهٔ پراکندگی آن همهٔ جهان را زیر پوشش خود قرارداد. با شکل‌گیری سازمان فیفا در سال ۱۹۰۴ و ایجاد رقابت جام جهانی فوتبال، این ورزش بسیار گسترده‌تر شد. طول زمین فوتبال ۹۰ تا ۱۲۰ متر و عرض آن ۴۵ تا ۹۰ متر است. مسابقهٔ فوتبال در دو نیمهٔ ۴۵دقیقه‌ای انجام می‌گیرد و در بازی‌های حذفی و رفت و برگشتی، گاهی، در صورت تساویِ نتیجه، وقت‌های اضافه و پنالتی نیز به آن افزوده می‌شود.,در فوتبال هر تیم باید چه تعداد بازیکن درون زمین داشته باشد؟,"{'text': ['یازده بازیکن'], 'answer_start': [191]}"
2,1218,مکانیک کوانتومی,مکانیک کوانتومی شاخه‌ای بنیادی از فیزیک نظری است که با پدیده‌های فیزیکی در مقیاس میکروسکوپی سر و کار دارد. در این مقیاس، کُنِش‌های فیزیکی در حد و اندازهٔ ثابت پلانک هستند. مقدار عددی ثابت پلانک نیز بسیار کوچک و برابر است با ۶٫۶۲۶x۱۰-۳۴. ژول-ثانیه. بنیادی‌ترین تفاوت مکانیک کوانتومی با مکانیک کلاسیک در این است که مکانیک کوانتومی توصیفی سازگار با آزمایش‌ها از ذرات در اندازه‌های اتمی و زیراتمی در اختیار می‌دهد، در حالی که مکانیک کلاسیک در قلمرو میکروسکوپی به نتایج نادرست می‌انجامد. زیرا در مقیاس‌های اتمی و زیراتمی که این نظریه‌ها با شکست مواجه می‌شوند، با دقت زیادی بسیاری از پدیده‌ها را توصیف می‌کند. مکانیک کوانتومی به همراه نسبیت پایه‌های فیزیک نوین را تشکیل می‌دهند. مکانیک کوانتومی یا نظریهٔ کوانتومی شامل نظریه‌ای دربارهٔ ماده و تابش الکترومغناطیسی و برهمکنش میان ماده و این تابش است. پایه‌های مکانیک کوانتومی در نیمهٔ اول قرن بیستم به کوشش ورنر هایزنبرگ، ماکس پلانک، آلبرت اینشتین، لویی دو بروی، نیلز بور، اروین شرودینگر، ماکس برن، جان فون نویمان، پل دیراک، ولفگانگ پاولی،ریچارد فاینمن و دیگران ساخته شد.,ورنرهایزنبرگ که بود؟,"{'text': [], 'answer_start': []}"
3,2265,داستان مصور,داستان مصور یا پی‌نما یا کُمیک استریپ نقاشی، یا مجموعه‌ای از نقاشی‌های دنباله‌دار است که ماجرایی را روایت می‌کند. کمیک معمولاً توسط یک تیم دونفره که یکی داستان را می‌نویسد و دیگری تصویرگری و جوهر زنی می‌کند، ساخته می‌شود. شرح روایت و بیان ماجراها به زبان تصویر و بعضاً با استفاده از دیالوگ‌ها، به زمان مصر باستان برمی‌گردد. اما اگر شرح روایت‌های مدرن را در نظر بگیریم، داستان‌های مصور به شکل کنونی آن نخستین بار در اروپا شکل گرفتند. در سده ۱۸ میلادی، از طریق داستان‌های مصور، به شکلی «هجوآمیز» از ولنگاری جنسی و فساد در دودمان بوربون، انتقاد می‌شد. نخستین کمیک‌های آمریکایی در سده بیستم میلادی شکل گرفتند. اولین کتاب کمیک بر اساس مجموعه‌ای از کمیک استریپ‌های روزنامه‌ای شکل گرفت. همان‌طور که از نام ک

## Preprocessing the training data

In [8]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/684 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

In [9]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

In [29]:
tokenizer("اسم شما چیست؟", "اسمم حسامه.")

{'input_ids': [2, 2415, 136, 1471, 117, 3, 31184, 7376, 33, 7, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [10]:
max_length = 384 # The maximum length of a feature (question and context)
doc_stride = 128 # The authorized overlap between two part of the context when splitting it is needed.

In [18]:
for i, example in enumerate(datasets["train"]):
    if len(tokenizer(example["question"], example["context"])["input_ids"]) > 384:
        break
example = datasets["train"][i]

In [19]:
len(tokenizer(example["question"], example["context"])["input_ids"])

390

In [20]:
len(tokenizer(example["question"], example["context"], max_length=max_length, truncation="only_second")["input_ids"])

384

In [21]:
tokenized_example = tokenizer(
    example["question"],
    example["context"],
    max_length=max_length,
    truncation="only_second",
    return_overflowing_tokens=True,
    stride=doc_stride
)

In [22]:
[len(x) for x in tokenized_example["input_ids"]]

[384, 146]

In [23]:
for x in tokenized_example["input_ids"][:2]:
    print(tokenizer.decode(x))

[CLS] در کشور ما به تلفن همراه چه میگویند؟[SEP] تلفن همراه یا گوشی همراه که به ان موبایل نیز گفته می شود ، وسیله ای برای ارسال و دریافت صدا از طریق ارتباط رادیویی در پهنای وسیع جغرافیایی است. منظور از موبایل وسیله ای است که برای اتصال به شبکه تلفن همراه به کار می رود. اولین گوشی تلفن بی سیم در سال ۱۹۷۳ با وزنی نزدیک به ۲ کیلوگرم ساخته شد. موبایل نسل های گوناگونی دارد و توسط شرکت های سازنده گوناگونی تولید می شود و به فروش می رسد. برخی از شرکت های بزرگ تولیدکننده گوشی همراه در دنیا شامل نوکیا، هواوی، سامسونگ الکترونیکس، ال جی الکترونیکس، موتورولا، سونی، اپل، شیا<unk>ومی، اچ تی سی، گوگل پیکسل و..... هستند. در تلفن های همراه به طور معمول یک مجموعه نرم افزار یا سیستم عامل برای کنترل سخت افزار به کار می رود و برنامه های جانبی توسط سیستم عامل اجرا می شوند. از سیستم های عامل معروف برای تلفن همراه، می توان به اندروید، ای اواس و ویندوز فون اشاره کرد. بررسی ها در زمینه میزان استفاده از تلفن های همراه نشان می دهد که تعداد زیادی از مردم در حالی به خواب می روند که تلفن همراه خود را در دست دارند یا ا

In [37]:
tokenized_example = tokenizer(
    example["question"],
    example["context"],
    max_length=max_length,
    truncation="only_second",
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
    stride=doc_stride
)
print(tokenized_example["offset_mapping"][0][:100])

[(0, 0), (0, 2), (2, 7), (7, 10), (10, 13), (13, 18), (18, 24), (24, 27), (27, 35), (35, 36), (0, 0), (0, 4), (4, 10), (10, 13), (13, 18), (18, 24), (24, 27), (27, 30), (30, 33), (33, 40), (40, 44), (44, 49), (49, 52), (52, 56), (56, 58), (58, 64), (64, 67), (67, 72), (72, 78), (78, 80), (80, 87), (87, 91), (91, 94), (94, 99), (99, 106), (106, 114), (114, 117), (117, 123), (123, 128), (128, 138), (138, 142), (142, 143), (143, 149), (149, 152), (152, 159), (159, 165), (165, 168), (168, 172), (172, 175), (175, 180), (180, 186), (186, 189), (189, 194), (195, 200), (200, 206), (206, 209), (209, 213), (213, 216), (216, 220), (220, 221), (221, 227), (227, 232), (232, 237), (237, 240), (240, 244), (244, 247), (247, 251), (251, 256), (256, 259), (259, 264), (264, 270), (270, 273), (273, 275), (275, 283), (283, 289), (289, 292), (292, 293), (293, 300), (300, 304), (304, 308), (308, 316), (316, 317), (317, 322), (322, 324), (324, 329), (329, 334), (334, 338), (338, 345), (345, 353), (353, 354), 

In [38]:
first_token_id = tokenized_example["input_ids"][0][1]
offsets = tokenized_example["offset_mapping"][0][1]
print(tokenizer.convert_ids_to_tokens([first_token_id])[0], example["question"][offsets[0]:offsets[1]])

▁در در


In [39]:
sequence_ids = tokenized_example.sequence_ids()
print(sequence_ids)

[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [ ]:
answers = example["answers"]
start_char = answers["answer_start"][0]
end_char = start_char + len(answers["text"][0])

# Start token index of the current span in the text.
token_start_index = 0
while sequence_ids[token_start_index] != 1:
    token_start_index += 1

# End token index of the current span in the text.
token_end_index = len(tokenized_example["input_ids"][0]) - 1
while sequence_ids[token_end_index] != 1:
    token_end_index -= 1

# Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
offsets = tokenized_example["offset_mapping"][0]
if (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
    # Move the token_start_index and token_end_index to the two ends of the answer.
    # Note: we could go after the last offset if the answer is the last word (edge case).
    while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
        token_start_index += 1
    start_position = token_start_index - 1
    while offsets[token_end_index][1] >= end_char:
        token_end_index -= 1
    end_position = token_end_index + 1
    print(start_position, end_position)
else:
    print("The answer is not in this feature.")

In [41]:
print(tokenizer.decode(tokenized_example["input_ids"][0][start_position: end_position+1]))
print(answers["text"][0])

موبایل
موبایل


In [11]:
pad_on_right = tokenizer.padding_side == "right"

In [12]:
def prepare_train_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

This function works with one or several examples. In the case of several examples, the tokenizer will return a list of lists for each key:

In [13]:
features = prepare_train_features(datasets['train'][:5])

In [14]:
tokenized_datasets = datasets.map(prepare_train_features, batched=True, remove_columns=datasets["train"].column_names)

Parameter 'function'=<function prepare_train_features at 0x7f5ff3f97ef0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/10 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

## Fine-tuning Albert on PersianQA

In [15]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/69.7M [00:00<?, ?B/s]

Some weights of the model checkpoint at m3hrdadfi/albert-fa-base-v2 were not used when initializing AlbertForQuestionAnswering: ['sop_classifier.classifier.weight', 'predictions.dense.weight', 'predictions.LayerNorm.bias', 'sop_classifier.classifier.bias', 'predictions.decoder.weight', 'predictions.dense.bias', 'predictions.LayerNorm.weight', 'predictions.decoder.bias', 'predictions.bias']
- This IS expected if you are initializing AlbertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of AlbertForQuestionAnswering were not initialized from the model checkpoint at m3hrdadfi/albert-fa-base-v2

In [16]:
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    output_dir=f"{model_name}-finetuned-persian_qa",
    save_strategy='steps',
    save_steps=50,
    evaluation_strategy = "epoch",
    report_to="wandb",  # enable logging to W&B
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=1,
    weight_decay=0.01,
    push_to_hub=True,
)

In [17]:
from transformers import default_data_collator

data_collator = default_data_collator

In [47]:
from google.colab import drive
drive.mount('/content/drive')

Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7f5f09e75e10>> (for pre_run_cell):


--- Logging error ---
Traceback (most recent call last):
  File "/usr/lib/python3.7/logging/__init__.py", line 1029, in emit
    self.flush()
  File "/usr/lib/python3.7/logging/__init__.py", line 1009, in flush
    self.stream.flush()
OSError: [Errno 107] Transport endpoint is not connected
Call stack:
  File "/usr/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.7/dist-packages/traitlets/config/application.py", line 846, in launch_instance
    app.start()
  File "/usr/local/lib/python3.7/dist-packages/ipykernel/kernelapp.py", line 499, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.7/dist-packages/tornado/platform/asyncio.py", line 132, in start
    self.asyncio_loop.run_forever()
  File "/usr/li

Exception: ignored

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7f5f09e75e10>> (for post_run_cell):


--- Logging error ---
Traceback (most recent call last):
  File "/usr/lib/python3.7/logging/__init__.py", line 1029, in emit
    self.flush()
  File "/usr/lib/python3.7/logging/__init__.py", line 1009, in flush
    self.stream.flush()
OSError: [Errno 107] Transport endpoint is not connected
Call stack:
  File "/usr/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.7/dist-packages/traitlets/config/application.py", line 846, in launch_instance
    app.start()
  File "/usr/local/lib/python3.7/dist-packages/ipykernel/kernelapp.py", line 499, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.7/dist-packages/tornado/platform/asyncio.py", line 132, in start
    self.asyncio_loop.run_forever()
  File "/usr/li

Exception: ignored

In [46]:
%cd drive/MyDrive/AUT/nlp/

Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7f5f09e75e10>> (for pre_run_cell):


--- Logging error ---
Traceback (most recent call last):
  File "/usr/lib/python3.7/logging/__init__.py", line 1029, in emit
    self.flush()
  File "/usr/lib/python3.7/logging/__init__.py", line 1009, in flush
    self.stream.flush()
OSError: [Errno 107] Transport endpoint is not connected
Call stack:
  File "/usr/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.7/dist-packages/traitlets/config/application.py", line 846, in launch_instance
    app.start()
  File "/usr/local/lib/python3.7/dist-packages/ipykernel/kernelapp.py", line 499, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.7/dist-packages/tornado/platform/asyncio.py", line 132, in start
    self.asyncio_loop.run_forever()
  File "/usr/li

Exception: ignored

/content/drive/MyDrive/AUT/nlp
Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7f5f09e75e10>> (for post_run_cell):


--- Logging error ---
Traceback (most recent call last):
  File "/usr/lib/python3.7/logging/__init__.py", line 1029, in emit
    self.flush()
  File "/usr/lib/python3.7/logging/__init__.py", line 1009, in flush
    self.stream.flush()
OSError: [Errno 107] Transport endpoint is not connected
Call stack:
  File "/usr/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.7/dist-packages/traitlets/config/application.py", line 846, in launch_instance
    app.start()
  File "/usr/local/lib/python3.7/dist-packages/ipykernel/kernelapp.py", line 499, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.7/dist-packages/tornado/platform/asyncio.py", line 132, in start
    self.asyncio_loop.run_forever()
  File "/usr/li

Exception: ignored

In [20]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

/content/drive/MyDrive/AUT/nlp/albert-fa-base-v2-finetuned-persian_qa is already a clone of https://huggingface.co/hdamghanian/albert-fa-base-v2-finetuned-persian_qa. Make sure you pull the latest changes with `repo.git_pull()`.


In [21]:
trainer.train('/content/drive/MyDrive/AUT/nlp/albert-fa-base-v2-finetuned-persian_qa/checkpoint-300')

Loading model from /content/drive/MyDrive/AUT/nlp/albert-fa-base-v2-finetuned-persian_qa/checkpoint-300.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 9101
  Num Epochs = 1
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 569
  Continuing training from checkpoint, will skip to saved global_step
  Continuing training from epoch 0
  Continuing training from global step 300
  Will skip the first 0 epochs then the first 300 batches in the first epoch. If this takes a lot of time, you can add the `--ignore_data_skip` flag to your launch command, but you will res

  0%|          | 0/300 [00:00<?, ?it/s]

Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Epoch,Training Loss,Validation Loss


Saving model checkpoint to albert-fa-base-v2-finetuned-persian_qa/checkpoint-350
Configuration saved in albert-fa-base-v2-finetuned-persian_qa/checkpoint-350/config.json
Model weights saved in albert-fa-base-v2-finetuned-persian_qa/checkpoint-350/pytorch_model.bin
tokenizer config file saved in albert-fa-base-v2-finetuned-persian_qa/checkpoint-350/tokenizer_config.json
Special tokens file saved in albert-fa-base-v2-finetuned-persian_qa/checkpoint-350/special_tokens_map.json
tokenizer config file saved in albert-fa-base-v2-finetuned-persian_qa/tokenizer_config.json
Special tokens file saved in albert-fa-base-v2-finetuned-persian_qa/special_tokens_map.json
Several commits (2) will be pushed upstream.
Saving model checkpoint to albert-fa-base-v2-finetuned-persian_qa/checkpoint-400
Configuration saved in albert-fa-base-v2-finetuned-persian_qa/checkpoint-400/config.json
Model weights saved in albert-fa-base-v2-finetuned-persian_qa/checkpoint-400/pytorch_model.bin
tokenizer config file saved

Epoch,Training Loss,Validation Loss
1,2.336500,2.106634


Saving model checkpoint to albert-fa-base-v2-finetuned-persian_qa/checkpoint-500
Configuration saved in albert-fa-base-v2-finetuned-persian_qa/checkpoint-500/config.json
Model weights saved in albert-fa-base-v2-finetuned-persian_qa/checkpoint-500/pytorch_model.bin
tokenizer config file saved in albert-fa-base-v2-finetuned-persian_qa/checkpoint-500/tokenizer_config.json
Special tokens file saved in albert-fa-base-v2-finetuned-persian_qa/checkpoint-500/special_tokens_map.json
Saving model checkpoint to albert-fa-base-v2-finetuned-persian_qa/checkpoint-550
Configuration saved in albert-fa-base-v2-finetuned-persian_qa/checkpoint-550/config.json
Model weights saved in albert-fa-base-v2-finetuned-persian_qa/checkpoint-550/pytorch_model.bin
tokenizer config file saved in albert-fa-base-v2-finetuned-persian_qa/checkpoint-550/tokenizer_config.json
Special tokens file saved in albert-fa-base-v2-finetuned-persian_qa/checkpoint-550/special_tokens_map.json
***** Running Evaluation *****
  Num examp

TrainOutput(global_step=569, training_loss=1.1007120839740774, metrics={'train_runtime': 19595.9611, 'train_samples_per_second': 0.464, 'train_steps_per_second': 0.029, 'total_flos': 150738519255552.0, 'train_loss': 1.1007120839740774, 'epoch': 1.0})

In [ ]:
trainer.push_to_hub(commit_message='finished training')

In [51]:
trainer.save_model("albert-persianQA")

Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7f5f09e75e10>> (for pre_run_cell):


--- Logging error ---
Traceback (most recent call last):
  File "/usr/lib/python3.7/logging/__init__.py", line 1029, in emit
    self.flush()
  File "/usr/lib/python3.7/logging/__init__.py", line 1009, in flush
    self.stream.flush()
OSError: [Errno 107] Transport endpoint is not connected
Call stack:
  File "/usr/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.7/dist-packages/traitlets/config/application.py", line 846, in launch_instance
    app.start()
  File "/usr/local/lib/python3.7/dist-packages/ipykernel/kernelapp.py", line 499, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.7/dist-packages/tornado/platform/asyncio.py", line 132, in start
    self.asyncio_loop.run_forever()
  File "/usr/li

Exception: ignored

Saving model checkpoint to albert-persianQA
Configuration saved in albert-persianQA/config.json
Model weights saved in albert-persianQA/pytorch_model.bin
tokenizer config file saved in albert-persianQA/tokenizer_config.json
Special tokens file saved in albert-persianQA/special_tokens_map.json
Saving model checkpoint to albert-fa-base-v2-finetuned-persian_qa
Configuration saved in albert-fa-base-v2-finetuned-persian_qa/config.json
Model weights saved in albert-fa-base-v2-finetuned-persian_qa/pytorch_model.bin
tokenizer config file saved in albert-fa-base-v2-finetuned-persian_qa/tokenizer_config.json
Special tokens file saved in albert-fa-base-v2-finetuned-persian_qa/special_tokens_map.json
Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Question Answering', 'type': 'question-answering'}, 'dataset': {'name': 'persian_qa', 'type': 'persian_qa', 'args': 'persian_qa'}}


Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7f5f09e75e10>> (for post_run_cell):


--- Logging error ---
Traceback (most recent call last):
  File "/usr/lib/python3.7/logging/__init__.py", line 1029, in emit
    self.flush()
  File "/usr/lib/python3.7/logging/__init__.py", line 1009, in flush
    self.stream.flush()
OSError: [Errno 107] Transport endpoint is not connected
Call stack:
  File "/usr/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.7/dist-packages/traitlets/config/application.py", line 846, in launch_instance
    app.start()
  File "/usr/local/lib/python3.7/dist-packages/ipykernel/kernelapp.py", line 499, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.7/dist-packages/tornado/platform/asyncio.py", line 132, in start
    self.asyncio_loop.run_forever()
  File "/usr/li

Exception: ignored

In [52]:
model.save_pretrained('backup')
tokenizer.save_pretrained('backup')

Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7f5f09e75e10>> (for pre_run_cell):


--- Logging error ---
Traceback (most recent call last):
  File "/usr/lib/python3.7/logging/__init__.py", line 1029, in emit
    self.flush()
  File "/usr/lib/python3.7/logging/__init__.py", line 1009, in flush
    self.stream.flush()
OSError: [Errno 107] Transport endpoint is not connected
Call stack:
  File "/usr/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.7/dist-packages/traitlets/config/application.py", line 846, in launch_instance
    app.start()
  File "/usr/local/lib/python3.7/dist-packages/ipykernel/kernelapp.py", line 499, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.7/dist-packages/tornado/platform/asyncio.py", line 132, in start
    self.asyncio_loop.run_forever()
  File "/usr/li

Exception: ignored

Configuration saved in backup/config.json
Model weights saved in backup/pytorch_model.bin
tokenizer config file saved in backup/tokenizer_config.json
Special tokens file saved in backup/special_tokens_map.json


('backup/tokenizer_config.json',
 'backup/special_tokens_map.json',
 'backup/spiece.model',
 'backup/added_tokens.json',
 'backup/tokenizer.json')

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7f5f09e75e10>> (for post_run_cell):


--- Logging error ---
Traceback (most recent call last):
  File "/usr/lib/python3.7/logging/__init__.py", line 1029, in emit
    self.flush()
  File "/usr/lib/python3.7/logging/__init__.py", line 1009, in flush
    self.stream.flush()
OSError: [Errno 107] Transport endpoint is not connected
Call stack:
  File "/usr/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.7/dist-packages/traitlets/config/application.py", line 846, in launch_instance
    app.start()
  File "/usr/local/lib/python3.7/dist-packages/ipykernel/kernelapp.py", line 499, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.7/dist-packages/tornado/platform/asyncio.py", line 132, in start
    self.asyncio_loop.run_forever()
  File "/usr/li

Exception: ignored